In [1]:
pip install PDFplumber

Note: you may need to restart the kernel to use updated packages.


In [350]:
import pandas as pd
import pdfplumber
import re
import os, os.path
from dateutil import parser
from sshtunnel import SSHTunnelForwarder
import sqlalchemy as db
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime, date


## Extracting available data

### Prior data
When the school provided those PDF documents, which were extracted from a free online form builder for school applications, my first action was to take a look in the very first pages of the first document and understand it. I realized that, instead of organized tables with information about the students, all data was stored in form-based PDFs.  It was very look alike a printed document filled by each student separately. The main goal of keeping those documents were just records of students inscriptions, with no strategic business intent for that data.

Therefore, I've proposed a different pipeline for that data, that not only could escalate, enabling larger volumes of data, but also that allowed the school to catalogue, clean, filter, manipulate and analyze all that value informations to find the best business solutions.


In [101]:
pdf_test = pdfplumber.open(".\\data\\211596498612667-0.pdf")
print(len(pdf_test.pages))
print(pdf_test.pages[0].extract_text())

30
Sunday, June 20, 2021
Fiche d'inscription 
Nom et prénom d'élève  Blatt Luce
Date de naissance 10 19 1960
Adresse Rue des vincennes, 9
Toulouse, 31500
E-mail marieluceblatt@gmail.com
Téléphone (0033) 607-103468
Cours:      
Horaire:   
Lundi      12h15 Barre à terre   
Lundi Heure
Mardi    9h Barre à terre   
Mardi Heure
          
Jeudi    9h30 pbt     
Jeudi Heure
Vendredi  10h classique moyen 
Vendredi Heure
      
            
Téléverser le Certi cat Médical
CamScanner 06-20-2021 20.39.pdf
pdf
1
Create your own automated PDFs with Jotform PDF Editor- It’s free


### Proposing a new way to store data

So, for a better comprehension of the available data, i've tried to create an organized table with all subscriptions information. Regardless any particularity of each student, they all filled the same form for submission, so I could easily identify the fields despite there was no obvious separators between fields.

The inscription form provides lots of information about each student. I created lists to store these informations, according to the fields filled.

In [103]:
def read_pdfs (pdf):
    """open the pdf file and extract all the text information corresponding to each field
    return info"""
    info = []
    for page in range(len(pdf.pages)):
        info.append(pdf.pages[page].extract_text())
    info = "".join(info).replace("Create your own automated PDFs with Jotform PDF Editor- It’s free","").split("Fiche d'inscription")
    for student in range(1,len(info)):
        try:
            nom = info[student].split("Nom et prénom d'élève")[1].split("\n")[0].strip()
        except:
            nom = 0
        try:
            naissance = info[student].split("Date de naissance")[1].split("\n")[0].strip()
        except:
            naissance = 0
        try:
            adresse = info[student].split("Adresse")[1].split("\n")[0].replace("\n"," ").strip()
        except:
            adresse = 0
        try:
            cite = info[student].split("Adresse")[1].split("\n")[1].split(",")[0].replace("\n"," ").strip()
        except:
            cite = 0
        try:
            postal = info[student].split("Adresse")[1].split("\n")[1].split(",")[1].split("\n")[0].replace("\n"," ").strip()
        except:
            postal = 0
        try:
            email = info[student].split("E-mail")[1].split("\n")[0].strip()
        except:
            email=0
        try:
            representant_legal = info[student].split("Représentant légal de l’inscrit (pour")[1].split("les mineurs")[0].strip()
        except:
            representant_legal = 0
        try:
            tel = info[student].split("Téléphone")[1].split("\n")[0].strip()
        except:
            tel = info[student].split("Téléphone")[1].split("\n")[0].strip()
        try:
            cours = info[student].split("Cours:")[1].split("Horaire:")[0].replace("\nCours", "").strip()
        except:
            cours = 0
        try:
            horaire = info[student].split("Horaire:")[1].split("Cours 2")[0].replace("\xa0", "").replace("Heure", "").strip()
            cours2 = info[student].split("Cours 2:")[1].split("Horaire:")[0].replace("\nCours", "").strip()
            try:
                horaire2 = info[student].split("Horaire:")[2].split("Cours 3")[0].replace("\xa0", "").replace("Heure", "").strip()
                cours3 = info[student].split("Cours 3:")[1].split("Horaire:")[0].replace("\nCours", "").strip()
                horaire3 = info[student].split("Horaire:")[3].split("Téléverser")[0].replace("Heure", "").strip()
            except: 
                horaire2= info[student].split("Horaire:")[2].split("Téléverser")[0].replace("\xa0", "").replace("Heure", "").strip()
                cours3 =0
                horaire3=0
        except:
            try:
                horaire = info[student].split("Horaire:")[1].split("Téléverser")[0].replace("\xa0", "").replace("Heure", "").strip()
            except:
                horaire = 0
            cours2 =0
            horaire2=0
            cours3 =0
            horaire3=0
        try:
            adhesion = info[student].split("\xa0\xa0\xa0\xa0\xa0+")[1].split("\n(valeur de chaque chèque)")[0].strip()
        except: 
            adhesion = info[student].split("Please Select")[1].split("adhésion")[0].strip() 
        try:
            paiement_fractionne =info[student].split("Par")[1].split(" cheques")[0].strip()
        except:
            paiement_fractionne =info[student].split("avec")[1].split("chèques")[0].strip()
        paiement_total = info[student].split("au total de")[1].split("€")[0].strip()
        name.append(nom)
        birthday.append(naissance)
        address.append(adresse)
        city.append(cite)
        pcode.append(postal)
        mail.append(email)
        telephone.append(tel)
        legal_representative.append(representant_legal)
        course.append(cours)
        schedule.append(horaire)
        course2.append(cours2)
        schedule2.append(horaire2)
        course3.append(cours3)
        schedule3.append(horaire3)
        registration.append(adhesion)
        installments.append(paiement_fractionne)
        total.append(paiement_total)

All available data was stored in the same folder, and the files' name differ only by the number at the end of them, from 0 to the last. 

In [104]:
name=[]
birthday=[]
address = []
city = []
pcode =[]
mail =[]
telephone =[]
legal_representative =[]
course=[]
schedule =[]
course2 =[]
schedule2 =[]
course3=[]
schedule3 =[]
registration =[]
installments =[]
total =[]
files = os.listdir('C:\\Users\\Tete\\Curso - DA\\Projeto Final\\data') 
for file in range(len(files)-1):
    pdf = pdfplumber.open(f".\\data\\211596498612667-{file}.pdf")
    read_pdfs (pdf)

### Creating the dataframe
After extracting all the information avaialable on those forms, I've gathered them in a dataframe which columns are the fields from the submission form.

In [105]:
attitude = pd.DataFrame(zip(name,birthday, address, city, pcode, mail, telephone, legal_representative, course, schedule, course2, schedule2, course3, schedule3, registration, installments, total))
attitude.columns = ['name','birthday', 'address', 'city', 'pcode','mail', 'telephone', 'legal_representative', 'course', 'schedule', 'course2', 'schedule2', 'course3', 'schedule3', 'registration', 'installments', 'total']

## Transforming data   

### Checking columns:

In [106]:
#Dropping duplicates
attitude = attitude.drop_duplicates().reset_index(drop=True)

In [107]:
#Let's standardize it!
# Strings
attitude.name = [nom.title() for nom in attitude.name]
attitude.address = [adresse.title() for adresse in attitude.address]
attitude.city = [cite.title() for cite in attitude.city]
attitude.mail = [email.lower() for email in attitude.mail]
attitude.legal_representative = [representant.title() if representant != 0 else 0 for representant in attitude.legal_representative]
attitude.course = [cours.title() if cours != 0 else 0 for cours in attitude.course]
attitude.course2= [cours2.title() if cours2 != 0 else 0 for cours2 in attitude.course2]
attitude.course3= [cours3.title() if cours3 != 0 else 0 for cours3 in attitude.course2]

In [270]:
#Birthday column
issues = []
for naissance in attitude.birthday:
    try:
        date = parser.parse(naissance)
    except:
        issues.append(naissance)
print(issues)
naissances=[]
for naissance in attitude.birthday:
    if naissance != 0:
        date = parser.parse(naissance.replace('25 nivelbre 1947', '25-11-1947').replace('18 décembre 2012', '18-12-2012').replace('17 août 2008', '17-08-2008').replace('4 juin 1975', '04-06-1975').replace('1er mai 2014', '01-05-2014').replace('18 AOUT 2010', '18-08-2010').replace('23 JANVIER 2017', '23-01-2017').replace('30061986', '30-06-1986').replace('17 juin 2016', '17-06-2016'))
        naissances.append(date.strftime('%d-%m-%Y'))
    else:
        naissances.append(0)
attitude.birthday = naissances
attitude['birthday'].iloc[57] = "13-03-1953"

[0, 0]


In [359]:
age=[]
ages = 0
for naissance in attitude.birthday:
    year=str(naissance).split('-')[-1]
    ages+=(int(year))

mean_year = ages/99
mean_year
for naissance in attitude.birthday:
    if naissance != 0:
        born_year = str(naissance).split('-')[-1]
        today = datetime.today()
        age.append(today.year - int(born_year))
    else:
        born_year = mean_year
        today = datetime.today()
        age.append(today.year - born_year)
        

In [122]:
#Column Telephone:
for row in range(len(attitude.telephone)):
    attitude['telephone'].iloc[row] = attitude['telephone'].iloc[row].replace('(33)', '+33').replace('(0033)', '+33').replace('(033)', '+33').replace('(','').replace("(0687026502)", "0687026502").replace("(Portable)", "").replace("(0634121580) 063-4121580", "063-4121580").replace("(0687026502) 068-7026502", "068-7026502").replace("(0689289829) ¨", "0689289829").replace("(0689289829) 068-9289829", "0689289829").replace("(0607991130) 060-7991130", "0607991130").replace("(0683365627) 068-3365627", "0683365627").replace("0689289829 ¨", '0689289829').replace('.',"").replace(')',"").replace("-","")

In [258]:
attitude['city'].iloc[59] = "Toulouse"
attitude['pcode'].iloc[59] = "31400"
attitude['city'].iloc[82] = "Toulouse"

In [200]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geolocalização")
latlong =[]
for row in range(len(attitude.address)):
    try:
        adresse = f'{attitude.address.iloc[row]}, {attitude.city.iloc[row]}, France, {attitude.pcode.iloc[row]}'
        location = geolocator.geocode(adresse)
        latlong.append(str(location.latitude, location.longitude))
    except:
        latlong.append(attitude.address.iloc[row])

In [229]:
lat=[]
long=[]
for lat_long in latlong:
    lat.append(float(lat_long.split(',')[0])) 
    long.append(float(lat_long.split(',')[1]))

In [259]:
toulouse=[]
for ville in attitude_eleves.city:
    if ville =='Toulouse':
        toulouse.append("Toulouse")
    else:
        toulouse.append("Autre ville")

In [375]:
def schedules (schedule_num):
    issues_horaire =[]
    for row in range(len(attitude[schedule_num])):
        if attitude[schedule_num].iloc[row] != 0:
            try:
                horaire = attitude[schedule_num].iloc[row].replace('\n1',' ').strip().split('\n')
                if len(horaire) == 2:
                    attitude[schedule_num].iloc[row] = attitude[schedule_num].iloc[row].replace('\n1',' ').strip().split('\n')[0].lower()
                else:
                    issues_horaire.append(row)
            except:
                continue
        else:
            continue
    return issues_horaire

def schedule_course (schedule_num, course_num):
    for row in range(len(attitude[schedule_num])):
        if attitude[schedule_num].iloc[row] == 'mardi 19h' or attitude[schedule_num].iloc[row] == 'mardi 18h' or attitude[schedule_num].iloc[row] =='vendredi 19h' or attitude[schedule_num].iloc[row] =='lundi 10h15':
            attitude[course_num].iloc[row] = 'Classique Moyen'
        elif attitude[schedule_num].iloc[row] == 'lundi 12h15' or attitude[schedule_num].iloc[row] == 'mardi 9h' or attitude[schedule_num].iloc[row] == 'samedi 12h15' or attitude[schedule_num].iloc[row] == 'samedi 12h' or attitude[schedule_num].iloc[row] == 'mardi 10h':
            attitude[course_num].iloc[row] = 'Barre à Terre'
        elif attitude[schedule_num].iloc[row] == 'mercredi 14h15':
            attitude[course_num].iloc[row] == 'Classique 1'
        elif attitude[schedule_num].iloc[row] == 'lundi 17h':
            attitude[course_num].iloc[row] = 'Préparatoire'    
        elif attitude[schedule_num].iloc[row] == 'mercredi 16h30' or attitude[schedule_num].iloc[row] == 'vendredi 20h30':
            attitude[course_num].iloc[row] = 'Pointes'
        elif attitude[schedule_num].iloc[row] == 'mardi 17h':
            attitude[course_num].iloc[row] = 'Éveil'
        elif attitude[schedule_num].iloc[row] == 'jeudi 17h10' or attitude[schedule_num].iloc[row] == 'jeudi 17h15':
            attitude[course_num].iloc[row] = 'Initiation'
        elif attitude[schedule_num].iloc[row] == 'mercredi 17h45':
            attitude[course_num].iloc[row] = 'Classique 2'
        elif attitude[schedule_num].iloc[row] == 'mercredi 14h25' or attitude[schedule_num].iloc[row] =='mercredi 14h15' or attitude[schedule_num].iloc[row] =='mercredi 13h15':
            attitude[course_num].iloc[row] = 'Classique 1'
        elif attitude[schedule_num].iloc[row] == 'lundi 18h':
            attitude[course_num].iloc[row] = 'Contemporain'
        elif attitude[schedule_num].iloc[row] == 'jeudi 20h' or attitude[schedule_num].iloc[row] =='lundi 20h30' or attitude[schedule_num].iloc[row] =='mercredi 9h' or attitude[schedule_num].iloc[row] =='jeudi 19h30':
            attitude[course_num].iloc[row] = 'Pilates'
        elif attitude[schedule_num].iloc[row] == 'lundi 10h':
            attitude[course_num].iloc[row] = 'Classique Moyen'
        elif attitude[schedule_num].iloc[row] == 'lundi 18h' or attitude[schedule_num].iloc[row] == 'mercredi 15h30':
            attitude[course_num].iloc[row] = 'Pbt'
        elif attitude[schedule_num].iloc[row] == 'jeudi 18h30' or attitude[schedule_num].iloc[row] == 'lundi 19h' or attitude[schedule_num].iloc[row] =='mercredi 19h30' or attitude[schedule_num].iloc[row] == 'samedi 10h40' or attitude[schedule_num].iloc[row] == 'samedi 10h30' or attitude[schedule_num].iloc[row] == 'lundi 19h15' or attitude[schedule_num].iloc[row] == 'mardi 19h30' or attitude[schedule_num].iloc[row] == 'mardi 19h15' or attitude[schedule_num].iloc[row] == 'jeudi 18h30':
            attitude[course_num].iloc[row] = 'Classique Interm. – Avancé'
        elif attitude[schedule_num].iloc[row] == 'vendredi 10h15' or attitude[schedule_num].iloc[row] =='vendredi 10h' or attitude[schedule_num].iloc[row] =='vendredi 19h' or attitude[schedule_num].iloc[row] =='lundi 10h' or attitude[schedule_num].iloc[row] =='lundi 19h15':
            attitude[course_num].iloc[row] = 'Classique Moyen'
        elif attitude[schedule_num].iloc[row] == 'lundi 19h30':
            attitude[course_num].iloc[row] = 'Classique Avancé'
        elif attitude[schedule_num].iloc[row] == 'vendredi 18h':
            attitude[course_num].iloc[row] = 'Moderne'
        elif attitude[schedule_num].iloc[row] == 'jeudi 9h30':
            attitude[course_num].iloc[row] = 'Pbt + Ballet Fitness'    

In [376]:
issues_horaire = schedules ('schedule')
issues_horaire

[0, 5, 11, 20, 25, 33, 58, 72, 89]

In [377]:
attitude['schedule'].iloc[0] = "mardi 12h15"
attitude['schedule2'].iloc[0] = "jeudi 9h30"
attitude['schedule'].iloc[5] = "Lundi, 12h15"
attitude['schedule2'].iloc[5] = "mardi 18h  vendredi 19h"
attitude['course'].iloc[11] = "barre à rerre"
attitude['schedule'].iloc[11] = "lundi 12h15 mardi 9h"
attitude['schedule'].iloc[20] = "lundi 19h15"
attitude['schedule2'].iloc[20] = "jeudi 18h30 vendredi 20h30"
attitude['schedule'].iloc[25] = "lundi 12h15"
attitude['schedule2'].iloc[25] = "vendredi 19h"
attitude['schedule'].iloc[28] ='mercredi 15h30'
attitude['schedule'].iloc[33] = "mardi 9h"
attitude['schedule2'].iloc[33] = "vendredi 10h15"
attitude['course'].iloc[58] = "classique avancé"
attitude['schedule'].iloc[58] = "mardi 19h15 jeudi 18h30"
attitude['course2'].iloc[58] = "pbt"
attitude['schedule2'].iloc[58] = "mercredi 15h30"

In [378]:
attitude.schedule.unique()
for row in range(len(attitude.schedule)):
    if attitude['schedule'].iloc[row] != 0:
        attitude['schedule'].iloc[row] = attitude['schedule'].iloc[row].lower().strip().replace('10h30\n\n\n\n\n\nsamedi \nsamedi','samedi 10h30').replace('lundi 18', 'lundi 18h').replace('lundi 17', 'lundi 17h').replace('lundi 17hh', 'lundi 17h').replace('lundi 18hh', 'lundi 18h').replace('mardi 10', 'mardi 10h').replace('mardi 10hh', 'mardi 10h').replace('mardi 17', 'mardi 17h').replace('mardi 17hh', 'mardi 17h').replace('mercredi 9', 'mercredi 9h').replace('mercredi 9hh', 'mercredi 9h').replace(',', '').replace(':', 'h').replace('h00', 'h').replace('\n1', '').replace(' h ', 'h').replace('vendredi 19h-20h30', 'vendredi 19h').replace('mardi 18h/19h30', 'mardi 18h').replace('mardi 18h-19h30', 'mardi 18h').replace('mercredi 16h30/17h30','mercredi 16h30').replace('mercredi 17h45-19h15','mercredi 17h45').replace('mardi 10 ', 'mardi 10h').replace('jeudi 17h15 - 18h15','jeudi 17h15').replace('lundi 17 ', 'lundi 17h').replace('jeudi 17h15-18h15', 'jeudi 17h15').replace('mercredi 14h15-15h30','mercredi 14h15').replace('mardi 17h - 17h45', 'mardi 17h').replace('jeudi 19h30-21h', 'jeudi 19h30').replace('lundi 17-18h', 'lundi 17h').replace('jeudi 20h - 21h', 'jeudi 20h').replace('lundi 10h11h30', 'lundi 10h').replace('lundi 18 ','lundi 18h').replace('lundi 19h 15 et 20h45', 'lundi 19h15').replace('mercredi 16.30/17.30', 'mercredi 16h30').replace('mardi 9.00/10.00', 'mardi 9h').replace('jeudi 18h30 - 20h', 'jeudi 18h30').replace('mardi 19h30 - 20h45','mardi 19h30').replace('10h30\n\n\n\n\n\nsamedi \nsamedi\n\n1', 'samedi 10h30').replace('mardi 12h15','lundi 12h15').replace('mardi 17 ','mardi 17h').replace('mercredi 9 ', 'mercredi 9h')
attitude.schedule.unique()

array(['lundi 12h15', 'lundi 17h', 'mardi 19h', 'lundi 20h30',
       'mercredi 14h15', 'mardi 17h', 'mardi 18h', 'samedi 12h',
       'lundi 12h15 mardi 9h', 'vendredi 19h', 'mercredi 16h30',
       'lundi 19h15', 'jeudi 17h10', 'mercredi 17h45', 'mardi 19h30',
       'mercredi 15h30', 'lundi 10h15', 'samedi 10h30', 'jeudi 17h15',
       'mardi 10h', 'mardi 9h', 'mercredi', 'lundi 18h', 'jeudi 19h30', 0,
       'lundi 17h-18h', 'jeudi 20h', 'mercredi 13h15', 'lundi 10h',
       'mardi 19h15 jeudi 18h30', 'jeudi 18h30', 'mercredi 19h30',
       'vendredi 10h15', 'samedi 10h30\n', 'samedi', 'mercredi 9h',
       'jeudi', '', 'lundi', 'vendredi 18h'], dtype=object)

In [379]:
issues_horaire2 = schedules ('schedule2')
issues_horaire2

[0, 3, 5, 19, 20, 25, 33, 58]

In [381]:
attitude.schedule2.unique()
for row in range(len(attitude.schedule2)):
    if attitude['schedule2'].iloc[row] != 0:
        attitude['schedule2'].iloc[row] = attitude['schedule2'].iloc[row].replace(',', '').replace(':', 'h').replace('h00', 'h').replace(' h ', 'h').replace('Jeudi, 9h30', 'jeudi 9h30').replace('Mardi, 18h,  Vendredi, 19h', 'mardi 18h,  vendredi 19h').replace('vendredi 20h30-21h15','vendredi 20h30').replace('vendredi 18/19h','vendredi 18h').replace('mercredi 19h30-21h', 'mercredi 19h30').replace('Mercredi 17h45/19h15\nMercredi \n\n\n\n\n2','Mercredi 17h45').replace('Jeudi, 18h30, Vendredi, 20h30','jeudi 18h30, vendredi 20h30').replace('mercredi 17.45/19.15','mercredi 17h45').replace('mercredi 15h30 - 16h30', 'mercredi 15h30').lower()
attitude.schedule2.unique()

array(['jeudi 9h30', 0, '19h', 'vendredi 19h', 'mardi 18h  vendredi 19h',
       'mercredi 15h30', 'vendredi 20h30', 'samedi 12h15', 'vendredi 18h',
       'mercredi 19h30', 'mercredi 17h45', 'jeudi 18h30 vendredi 20h30',
       'samedi 10h30', 'mercredi 16h30', 'vendredi 10h15', 'lundi 20h45',
       'samedi 10h40', 'lundi 19h15', 'jeudi ', 'mercredi ', 'vendredi '],
      dtype=object)

In [382]:
issues_horaire3 = schedules ('schedule3')
issues_horaire3

[18, 20, 57, 58, 92]

In [383]:
attitude['schedule3'].iloc[18] = 'vendredi 20h30'
attitude['schedule3'].iloc[20] = 'vendredi 20h30'
attitude['schedule3'].iloc[57] = 'vendredi 10h'
attitude['schedule3'].iloc[58] = 'mercredi 17h45'
attitude['schedule3'].iloc[92] = 'samedi 10h30'

In [384]:
attitude.schedule3.unique()
for row in range(len(attitude.schedule3)):
    if attitude['schedule3'].iloc[row] != 0:
        attitude['schedule3'].iloc[row] = attitude['schedule3'].iloc[row].lower().replace(',', '').replace(':', 'h').replace('h00', 'h').replace(' h ', 'h').replace('vendredi \xa019h -20h30\xa0', 'vendredi 19h').replace('vendredi \xa019/20h30\xa0','vendredi 19h').replace('mercredi \xa017h45\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0', 'mercredi 17h45').replace('samedi \xa0\xa0\xa010h30\xa0\xa0\xa0', 'samedi 10h30').replace('\xa0', '')
attitude.schedule3.unique()

array([0, 'vendredi 19h', 'vendredi 20h30', 'mercredi 17h45',
       'samedi 10h30', 'vendredi 10h', 'mercredi ', 'vendredi '],
      dtype=object)

In [385]:
schedule_course ('schedule', 'course')
schedule_course ('schedule2', 'course2')
schedule_course ('schedule3', 'course3')

In [400]:
attitude[attitude['course']==0]
attitude['course'].iloc[39] = "pilates"
attitude['schedule'].iloc[39] = "juedi 20h"
attitude['course'].iloc[41] = 'classique 1'
attitude['course'].iloc[51] = 'classique 1'
attitude['course'].iloc[78] = 'classique 1'
attitude['course'].iloc[79] = 'classique moyen'
attitude['schedule'].iloc[79] = "vendredi 10h"
attitude['course'].iloc[79] = 'classique interm. – avancé'
attitude['schedule'].iloc[93] = "vendredi 10h"
attitude['course'].iloc[93] = 'classique interm. – avancé'
attitude['course'].iloc[46] = "pilates"
attitude['schedule'].iloc[46] = "lundi 9h mercredi 9h vendredi 9h"
attitude['course'].iloc[46] = "classique moyen"
attitude['schedule'].iloc[46] = "lundi 10h"
attitude['course'].iloc[46] = "pbt + ballet fitness"
attitude['schedule'].iloc[46] = "juedi 9h30"
attitude['course'].iloc[56] = "pilates"
attitude['schedule'].iloc[56] = "mercredi 9h vendredi 9h"
attitude['course'].iloc[66] = "pilates"
attitude['schedule'].iloc[66] = "lundi 9h"
attitude['course3'].iloc[81] = ""
attitude['schedule'].iloc[81] = "mercredi 15h30"
attitude['schedule2'].iloc[81] = "mercredi 16h30"
attitude['schedule3'].iloc[81] = "0"
attitude['course'].iloc[83] = "pilates"
attitude['schedule'].iloc[83] = "lundi 20h45"
attitude['course2'].iloc[83] = 0
attitude['schedule2'].iloc[83] = 0
attitude['course'].iloc[88] = "Classique Interm. – Avancé"
attitude['schedule'].iloc[88] = "jeudi 18h30"
attitude['schedule2'].iloc[88] = "vendredi 20h30"
attitude['course3'].iloc[88] = "0"
attitude['schedule3'].iloc[88] = "0"
attitude['course'].iloc[95] = "barre à terre"
attitude['schedule'].iloc[95] = "mardi 9h"
attitude['course'].iloc[57]='Classique Moyen'
attitude['course2'].iloc[57]='Classique Interm. – Avancé'
attitude['schedule2'].iloc[57]='jeudi 18h30'
attitude['course3'].iloc[57]='Classique Interm. – Avancé'
attitude['schedule3'].iloc[57]='vendredi 10h'

In [401]:
attitude.course.unique()
for row in range(len(attitude.course)):
    if attitude['course'].iloc[row] != 0:
        attitude['course'].iloc[row] = attitude['course'].iloc[row].replace("Eveil", 'éveil').replace("Classiquee Avancé", "Classique Avancé").replace('barre à rerre', 'barre à terre').replace('inter/ avance', 'classique interm. – avancé').lower()
        if attitude['course'].iloc[row] =='classique interm':
            attitude['course'].iloc[row] = ('classique interm. – avancé')
        if attitude['course'].iloc[row] =='1' or attitude['course'].iloc[row] =='classique':
            attitude['course'].iloc[row] = ("classique 1")
    elif attitude['course'].iloc[row] == "":
        attitude['course'].iloc[row] == ""
attitude.course.unique()

array(['barre à terre', 'préparatoire', 'classique moyen', 'pilates',
       'classique 1', 'éveil', 'pointes', 'classique interm. – avancé',
       'initiation', 'classique 2', 'pbt', 'contemporain',
       'pbt + ballet fitness', '', 'classique avancé', 'carte 10 cours',
       'moderne'], dtype=object)

In [402]:
attitude.course2.unique()
for row in range(len(attitude.course2)):
    if attitude['course2'].iloc[row] != 0:
        attitude['course2'].iloc[row] = attitude['course2'].iloc[row].replace("Classiquee Avancé", "Classique Avancé").replace('barre à rerre', 'barre à terre').replace('Inter/ Avance', 'classique interm. – avancé').lower()
        if attitude['course2'].iloc[row] =='barre':
            attitude['course2'].iloc[row] = ("barre à terre")
        if attitude['course2'].iloc[row] =='classique interm':
            attitude['course2'].iloc[row] = ('classique interm. – avancé')
        if attitude['course2'].iloc[row] =='classique':
            attitude['course2'].iloc[row] = ("classique 1")
    elif attitude['course2'].iloc[row] == "":
        attitude['course2'].iloc[row] == ""
attitude.course2.unique()

array(['pbt + ballet fitness', 0, '', 'classique moyen', 'pbt', 'pointes',
       'barre à terre', 'moderne', 'classique interm. – avancé',
       'classique 2'], dtype=object)

In [403]:
attitude.course3.unique()
for row in range(len(attitude.course3)):
    if attitude['course3'].iloc[row] != 0:
        attitude['course3'].iloc[row] = attitude['course3'].iloc[row].replace("Classiquee Avancé", "Classique Avancé").replace('barre à rerre', 'barre à terre').replace('inter/ avance', 'classique interm. – avancé').lower()
        if attitude['course3'].iloc[row] =='barre':
            attitude['course3'].iloc[row] = ("barre à terre")
        if attitude['course3'].iloc[row] =='classique':
            attitude['course3'].iloc[row] = ("classique 1")
    elif attitude['course3'].iloc[row] == "":
        attitude['course3'].iloc[row] == ""
attitude.course3.unique()

array([0, 'classique moyen', '', 'barre à terre', 'pbt', 'pointes',
       'classique 2', 'classique 1', 'classique interm. – avancé', '0'],
      dtype=object)

In [29]:
for row in range(len(attitude.registration)):
    attitude.registration.iloc[row] = '30€'

In [390]:
for row in range(len(attitude.installments)):
    attitude.installments.iloc[row] = attitude.installments.iloc[row].replace('shh', '1').replace('10 chéques\n(1-10)', '10').replace('9 + 1', '10')
attitude.installments.value_counts()
attitude.installments.iloc[34] ='3'
attitude.installments.iloc[46] ='1'
attitude.installments.value_counts()

3     39
1     36
10    14
4      5
2      3
5      2
Name: installments, dtype: int64

In [391]:
for row in range(len(attitude['total'])):
    if attitude['total'].iloc[row] == '+ 30':
        if attitude['course'].iloc[row] == 'carte 10 cours':
            attitude['total'].iloc[row] = 1000
        elif attitude['course2'].iloc[row] == 0:
            attitude['total'].iloc[row] = 500
        else:
            if attitude['course3'].iloc[row] == 0:
                attitude['total'].iloc[row] = 720
            else:
                attitude['total'].iloc[row] = 880
attitude['total'].iloc[46] = 1040
attitude['total'].iloc[53] = 720
attitude['total'].iloc[54] = 500
attitude['total'].iloc[58] = 1040
for row in range(len(attitude['total'])):
    attitude['total'].iloc[row] = int(attitude['total'].iloc[row])
print(attitude['total'].unique())

[720 500 880 1000 1040]


In [392]:
attitude_eleves = attitude.drop(columns=["course", "schedule",  "course2", "schedule2",  "course3", "schedule3", "registration", "installments", "total"])
attitude_cours22 = attitude.drop(columns=[ "birthday", "address", "city", "pcode", "mail", "telephone", "legal_representative",  "registration", "installments", "total"])
attitude_paiement22 = attitude.drop(columns=[ "birthday", "address", "city", "pcode", "mail", "telephone",  "legal_representative", "course", "schedule",  "course2", "schedule2",  "course3", "schedule3",])

In [393]:
attitude_eleves.insert(5, "lat", lat)
attitude_eleves.insert(6, "long", long)
attitude_eleves.insert(4, "toulouse", toulouse)
attitude_eleves.insert(2, "age", age)

In [406]:
courses = []
for cours in attitude_cours22['course'].unique():
    if cours != "0" and cours != 0 and cours != "":
        courses.append(cours)
for cours2 in attitude_cours22['course2'].unique():
    if cours2 != "0" and cours2 != 0 and cours != "":
        courses.append(cours2)
for cours3 in attitude_cours22['course3'].unique():
    if cours3 != '0' and cours3 != 0 and cours != "":
        courses.append(cours3)

courses = set(courses)
courses

{'',
 'barre à terre',
 'carte 10 cours',
 'classique 1',
 'classique 2',
 'classique avancé',
 'classique interm. – avancé',
 'classique moyen',
 'contemporain',
 'initiation',
 'moderne',
 'pbt',
 'pbt + ballet fitness',
 'pilates',
 'pointes',
 'préparatoire',
 'éveil'}

In [407]:
rows =[]
def students (course):
    cours =[]
    for row in range(len(attitude_cours22['course'])):
        if attitude_cours22['course'].iloc[row] == course or attitude_cours22['course2'].iloc[row] == course or attitude_cours22['course3'].iloc[row] == course:
            cours.append(attitude_cours22["name"].iloc[row])
    return cours

In [408]:
data_courses = dict()
for course in courses:
    try:
            data_courses[course]= students(course)
            print(course)
    except:
        continue
        
data_courses.keys()


éveil
préparatoire
carte 10 cours
pbt
pilates
pointes
contemporain
classique interm. – avancé
classique moyen
classique 1
classique avancé
initiation
moderne
pbt + ballet fitness
barre à terre
classique 2


dict_keys(['', 'éveil', 'préparatoire', 'carte 10 cours', 'pbt', 'pilates', 'pointes', 'contemporain', 'classique interm. – avancé', 'classique moyen', 'classique 1', 'classique avancé', 'initiation', 'moderne', 'pbt + ballet fitness', 'barre à terre', 'classique 2'])

In [39]:
for course in data_courses.keys():
    if course != 0 or course !='carte 10 cours':
        data_courses[course].append('Shawali Natacha')

In [40]:
attitude_cours23 = attitude_cours22.iloc[0:0]
attitude_paiement23 = attitude_paiement22.iloc[0:0]

In [410]:
colunes=[]
for row in range(len(attitude_cours22['course'])):
    rows=[]
    rows.append(attitude_cours22["name"].iloc[row])
    for course in data_courses.keys():
        if attitude_cours22['course'].iloc[row] == course or attitude_cours22['course2'].iloc[row] == course or attitude_cours22['course3'].iloc[row] == course:
            rows.append(1)
        else:
            rows.append(0)
    colunes.append(rows)
            
classes22 = pd.DataFrame(colunes)
classes22.columns = ('name', '', 'eveil', 'préparatoire', 'carte_10_cours', 'pbt', 'pilates', 'pointes', 'contemporain', 'classique_interm_avance', 'classique_moyen', 'classique_1', 'classique_avance', 'initiation', 'moderne', 'pbt_ballet_fitness', 'barre_a_terre', 'classiqu_2')
classes22 = classes22.drop(['','carte_10_cours'], axis=1)


In [411]:
classes23 = classes22.iloc[0:0]

In [412]:
attitude_eleves.to_csv('elevesdf.csv')
attitude_cours22.to_csv('coursdf22.csv')
classes22.to_csv('classes22.csv')
attitude_paiement22.to_csv('paiment22.csv')
attitude_cours23.to_csv('coursdf23.csv')
classes23.to_csv('classes23.csv')
attitude_paiement23.to_csv('paiment23.csv')

In [243]:
def open_ssh_tunnel_and_mysql ():
    """Open an SSH tunnel and connect to a MySQL server using the SSH tunnel connection
    return Global MySQL connection"""
    load_dotenv()
    db_server= os.getenv('db_server')
    user=os.getenv("user")
    db_port=os.getenv("db_port")
    password=os.getenv("password")
    ip=os.getenv("ip")
    db_name=os.getenv("db_name")
    ip_ssh=os.getenv("ip_ssh")
    ssh_username=os.getenv("ssh_username")
    ssh_password=os.getenv("ssh_password")
    remote_bind_address=os.getenv("remote_bind_address")
    global server
    global connection
    global conn_addr
    global engine
    try:    
        print('Opening tunnel :-P')  
        server = SSHTunnelForwarder((ip_ssh, 4242), ssh_username=ssh_username, ssh_password=ssh_password, remote_bind_address=(db_server, 3306))
        server.start()
        print('Tunnel opend :-P')
        port = str(server.local_bind_port)
        conn_addr = 'mysql://' + user + ':' + password + '@' + db_server + ':' + port + '/' + db_name
        engine = create_engine(conn_addr)
        connection = engine.connect()
        print('Yeah! MySQL server connected using the SSH tunnel connection!')
    except Exception as E:
        print(E)
    
def create_table (dataframe, table=str):
    dataframe.to_sql(table, conn_addr, if_exists='replace', index=False)
    print('All done, Madam!')
    
def drop_table(table):
    engine.execute(f"DROP table IF EXISTS {table}")
    print('Table deleted, Madam!')
    
def table_to_csv(table):
    csv = sql_query_to_csv


def disconnect_mysql ():
    """Disconnect from MySQL server"""
    connection.close()  
    print('MySQL server is not connected anymore!')
    
def shut_ssh_tunnel ():
    """Stop the SSH tunnel"""
    server.stop()
    print("You've stopped the SSH tunnel!")

In [413]:
open_ssh_tunnel_and_mysql ()
#create_table (attitude_eleves, 'elevesdf')
create_table (attitude_cours22, 'coursdf22')
create_table (classes22, 'classesdf22')
create_table (attitude_paiement22, 'paimentsdf22')
create_table (attitude_cours23, 'coursdf23')
create_table (classes23, 'classesdf23')
create_table (attitude_paiement23, 'paimentsdf23')
disconnect_mysql ()
shut_ssh_tunnel ()

Opening tunnel :-P
Tunnel opend :-P


Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\Tete\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 682, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\Tete\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 887, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\Tete\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 667, in do_rollback
    dbapi_connection.rollback()
MySQLdb._exceptions.OperationalError: (2006, 'Server has gone away')


Yeah! MySQL server connected using the SSH tunnel connection!
All done, Madam!
All done, Madam!
All done, Madam!
All done, Madam!
All done, Madam!
All done, Madam!
MySQL server is not connected anymore!
You've stopped the SSH tunnel!


In [365]:
attitude_cours22

,name,course,schedule,course2,schedule2,course3,schedule3
0,Blatt Luce,,Lundi 12h15 Barre à terre\nLundi \nMardi 9h Ba...,0,0,0,0
1,Paulon Lily,Classique,Lundi 17h\nLundi \n\n\n\n\n\n\n1,0,0,0,0
2,Mezard Emmanuelle,Barre Moyen,Mardi 19H\nMardi \n\n\n\n\n\n1,Pbt + Ballet Fitness,Jeudi 9H30\nJeudi,Pbt + Ballet Fitness,Vendredi 19h -20h30 \nVendredi
3,Corbiere Beatrix,,Lundi 20h30\nLundi \n\n\n\n\n\n\n1,,19h,,0
4,François Eve,Barre Au Sol,Lundi 12h15\nLundi \n\n\n\n\n\n\n1,Barre,Vendredi 19h\nVendredi,Barre,0
...,...,...,...,...,...,...,...
94,Peccia-Galletto Sasha,Débutant,Mardi 17H\nMardi,0,0,0,0
95,Galvani Francoise,0,0,0,0,0,0
96,Lemozit Sasha,Moderne,Vendredi 18h\nVendredi,0,0,0,0
97,Blanc Bouny Celeste,Danse Classique Debutant,Lundi 17H\nLundi,0,0,0,0


In [374]:
open_ssh_tunnel_and_mysql ()

engine.table_names()

Opening tunnel :-P
Tunnel opend :-P
Yeah! MySQL server connected using the SSH tunnel connection!


['classesdf22',
 'classesdf23',
 'coursdf22',
 'coursdf23',
 'elevesdf',
 'paimentsdf22',
 'paimentsdf23']